# Project Part One

In [ ]:
%%sql
CREATE TABLE hospital (
    hospital_pk TEXT PRIMARY KEY,
    zipcode INTEGER,
    hospital_name TEXT,
    address TEXT, 
    geocoded_hospital_address TEXT,
    type_of_hospital TEXT,
    type_of_ownership TEXT,
    emergency_services BOOLEAN,
    zipcodeid INTEGER NOT NULL REFERENCES locations(id),
);

In [ ]:
%%sql
CREATE TABLE weekly_logs (
    id SERIAL PRIMARY KEY,
    collection_week DATE,
    all_adult_hospital_beds_7_day_avg INTEGER,
    all_pediatric_inpatient_beds_7_day_avg INTEGER,
    all_adult_hospital_inpatient_bed_occupied_7_day_avg INTEGER,
    all_pediatric_inpatient_bed_occupied_7_day_avg INTEGER,
    total_icu_beds_7_day_avg INTEGER,
    icu_beds_used_7_day_avg INTEGER,
    inpatient_beds_used_covid_7_day_avg INTEGER,
    staffed_icu_adult_patients_confirmed_covid_7_day_avg INTEGER,
    hospital_pk TEXT NOT NULL REFERENCES hospital(hospital_pk)
);

In [ ]:
%%sql
CREATE TABLE hospital_quality (
    id SERIAL PRIMARY KEY,
    quality_rating NUMERIC,
    date_updated DATE REFERENCES weekly_logs(id),
    hospital_pk TEXT NOT NULL REFERENCES hospital(hospital_pk)
);

In [ ]:
%%sql
CREATE TABLE locations (
    zipcode INTEGER PRIMARY KEY,
    state TEXT,
    city TEXT,
    fips_code TEXT,
    unique(zipcode)
);